In [21]:
import torch
import torchvision

In [22]:
backbone = torchvision.models.resnet18(pretrained=True).to('cuda')

In [24]:
num_classes = 5
backbone.fc = torch.nn.Linear(backbone.fc.in_features, num_classes).to('cuda')

In [25]:
width = 800
height = 600

In [26]:
dataset_dir = "C:/Users/Silver/BaiduSyncdisk/CSOL 集成工具数据集/"
class RemoveAlpha:
    def __call__(self, img):
        if img.mode == 'RGBA':
            img = img.convert('RGB')
        return img
import random
transform = torchvision.transforms.Compose([
    RemoveAlpha(),
    torchvision.transforms.Pad(padding=50, fill=0, padding_mode='constant'),
    torchvision.transforms.Resize((height, width)),
    torchvision.transforms.RandomHorizontalFlip(p=0.5),
    torchvision.transforms.RandomVerticalFlip(p=0.5),
    torchvision.transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    torchvision.transforms.RandomErasing(p=0.5, scale=(0.02, 0.2), ratio=(0.3, 3.3), value=0),
])
dataset = torchvision.datasets.ImageFolder(dataset_dir, transform=transform)
test_set_ratio = 0.1
train_size = int(len(dataset) * (1 - test_set_ratio))
test_size = len(dataset) - train_size
train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])
train_loader = torch.utils.data.DataLoader(train_set, batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=16, shuffle=False)


In [27]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(backbone.parameters(), lr=0.0001)
num_epochs = 500

In [28]:
for epoch in range(num_epochs):
    backbone.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to('cuda')
        labels = labels.to('cuda')
        optimizer.zero_grad()
        outputs = backbone(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")
    backbone.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to('cuda')
            labels = labels.to('cuda')
            outputs = backbone(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = 100 * correct / total
    print(f"Test Accuracy: {acc:.2f}%")
    if epoch > 0 and epoch % 50 == 0:
        torch.save(backbone.state_dict(), f"CSOL-Utilities-ResNet18-800x600-epoch{epoch}.pth")


Epoch [1/500], Loss: 0.6916
Test Accuracy: 90.00%
Epoch [2/500], Loss: 0.2116
Test Accuracy: 90.00%
Epoch [3/500], Loss: 0.1348
Test Accuracy: 90.00%
Epoch [4/500], Loss: 0.0860
Test Accuracy: 95.00%
Epoch [5/500], Loss: 0.0632
Test Accuracy: 95.00%
Epoch [6/500], Loss: 0.0488
Test Accuracy: 95.00%
Epoch [7/500], Loss: 0.0619
Test Accuracy: 95.00%
Epoch [8/500], Loss: 0.0344
Test Accuracy: 95.00%
Epoch [9/500], Loss: 0.0340
Test Accuracy: 95.00%
Epoch [10/500], Loss: 0.0412
Test Accuracy: 95.00%
Epoch [11/500], Loss: 0.0234
Test Accuracy: 95.00%
Epoch [12/500], Loss: 0.0126
Test Accuracy: 95.00%
Epoch [13/500], Loss: 0.0466
Test Accuracy: 95.00%
Epoch [14/500], Loss: 0.0311
Test Accuracy: 97.50%
Epoch [15/500], Loss: 0.0251
Test Accuracy: 97.50%
Epoch [16/500], Loss: 0.0235
Test Accuracy: 97.50%
Epoch [17/500], Loss: 0.0520
Test Accuracy: 97.50%
Epoch [18/500], Loss: 0.0223
Test Accuracy: 97.50%
Epoch [19/500], Loss: 0.0317
Test Accuracy: 97.50%
Epoch [20/500], Loss: 0.0203
Test Accura

In [12]:
# 保存 .pth
torch.save(backbone.state_dict(), "CSOL-Utilities-ResNet18-800x600.pth")

In [13]:
# 导出 .onnx
dummy_input = (torch.randn(1, 3, height, width),)
torch.onnx.export(
    backbone.to('cpu'),
    dummy_input,
    "CSOL-Utilities-ResNet18-800x600.onnx",
    export_params=True, # Store trained parameters
    opset_version=11, # ONNX version
    do_constant_folding=True, # Optimize constant folding
    input_names=["screenshot"],
    output_names=["interface_type"],
)

C:\Users\Silver\AppData\Local\Temp\ipykernel_20636\2390526172.py:3: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(
